# MHW 2

> Выполнил: Разуваев Никита Сергеевич

## Содержание:

* [Задание 1](#task-1)
    * Результат: https://storage.yandexcloud.net/nsrazuvaevbucket/mhw2-top10-longest-sess.txt
* [Задание 2.1](#task-2-1)
    * Результат: https://storage.yandexcloud.net/nsrazuvaevbucket/mhw2-top10-categories.txt
* [Задание 2.2](#task-2-2)
    * Результат: https://storage.yandexcloud.net/nsrazuvaevbucket/mhw2-top10-words.txt
* [Задание 2.3](#task-2-3)
    * Результат: https://storage.yandexcloud.net/nsrazuvaevbucket/mhw2-mean-price.txt
* [Задание 2.4](#task-2-4)
    * Результат: https://storage.yandexcloud.net/nsrazuvaevbucket/mhw2-all-days-active-users.txt
* [Задание 2.5](#task-2-5)
    * Результат: https://storage.yandexcloud.net/nsrazuvaevbucket/mhw2-uniq-users-per-day.txt

## Задание 1 <a class="anchor" id="task-1"></a>

### Отправка данных в hdfs

Воспользуемся датасетом скачанным в мини-дз 1

In [6]:
%cd "~/storage/avito-context-ad-clicks"
! ls

/home/ubuntu/storage/avito-context-ad-clicks
VisitsStream.tsv


In [7]:
! head -n 10 VisitsStream.tsv

UserID	IPID	AdID	ViewDate
59703	1259356	469877	2015-04-25 00:00:00.0
154389	1846749	27252551	2015-04-25 00:00:00.0
218628	2108380	31685325	2015-04-25 00:00:00.0
231535	837110	18827716	2015-04-25 00:00:00.0
282306	1654210	29363673	2015-04-25 00:00:00.0
295068	601505	588324	2015-04-25 00:00:00.0
501897	158476	4103261	2015-04-25 00:00:00.0
655394	631692	9860544	2015-04-25 00:00:00.0
765603	804403	29475627	2015-04-25 00:00:00.0


Обрежем названия столбцов.

In [8]:
! sed -e '1'd VisitsStream.tsv > VisitsStreamNoHeader.tsv

In [9]:
! ls

VisitsStreamNoHeader.tsv  VisitsStream.tsv


Положим данные в hdsf:

In [10]:
! hdfs dfs -ls /

Found 5 items
drwx------   - mapred hadoop          0 2022-01-26 00:09 /hadoop
drwxr-xr-x   - hdfs   hadoop          0 2022-02-06 05:47 /system
drwxrwxrwt   - hdfs   hadoop          0 2022-01-26 00:09 /tmp
drwxrwxrwt   - hdfs   hadoop          0 2022-02-06 05:38 /user
drwxrwxrwt   - hdfs   hadoop          0 2022-01-26 00:09 /var


In [11]:
! hdfs dfs -rm -r /user/avito/data
! hdfs dfs -mkdir -p /user/avito/data

Deleted /user/avito/data


In [12]:
%%time
! hdfs dfs -put VisitsStreamNoHeader.tsv /user/avito/data/

CPU times: user 7.89 s, sys: 1.66 s, total: 9.55 s
Wall time: 8min 21s


In [13]:
! sudo hdfs balancer 

2022-02-06 22:23:21,935 INFO balancer.Balancer: namenodes  = [hdfs://rc1a-dataproc-m-ewr760zc7czy40ae.mdb.yandexcloud.net:8020]
2022-02-06 22:23:21,955 INFO balancer.Balancer: parameters = Balancer.BalancerParameters [BalancingPolicy.Node, threshold = 10.0, max idle iteration = 5, #excluded nodes = 0, #included nodes = 0, #source nodes = 0, #blockpools = 0, run during upgrade = false]
2022-02-06 22:23:21,955 INFO balancer.Balancer: included nodes = []
2022-02-06 22:23:21,955 INFO balancer.Balancer: excluded nodes = []
2022-02-06 22:23:21,955 INFO balancer.Balancer: source nodes = []
Time Stamp               Iteration#  Bytes Already Moved  Bytes Left To Move  Bytes Being Moved  NameNode
2022-02-06 22:23:21,958 INFO balancer.NameNodeConnector: getBlocks calls for hdfs://rc1a-dataproc-m-ewr760zc7czy40ae.mdb.yandexcloud.net:8020 will be rate-limited to 20 per second
org.apache.hadoop.ipc.RemoteException(org.apache.hadoop.security.AccessControlException): Access denied for user root. Super

In [14]:
! sudo chmod 0777 /usr/lib/hadoop/logs
! sudo -u hdfs hdfs balancer 

2022-02-06 22:23:27,506 INFO balancer.Balancer: namenodes  = [hdfs://rc1a-dataproc-m-ewr760zc7czy40ae.mdb.yandexcloud.net:8020]
2022-02-06 22:23:27,509 INFO balancer.Balancer: parameters = Balancer.BalancerParameters [BalancingPolicy.Node, threshold = 10.0, max idle iteration = 5, #excluded nodes = 0, #included nodes = 0, #source nodes = 0, #blockpools = 0, run during upgrade = false]
2022-02-06 22:23:27,509 INFO balancer.Balancer: included nodes = []
2022-02-06 22:23:27,509 INFO balancer.Balancer: excluded nodes = []
2022-02-06 22:23:27,509 INFO balancer.Balancer: source nodes = []
Time Stamp               Iteration#  Bytes Already Moved  Bytes Left To Move  Bytes Being Moved  NameNode
2022-02-06 22:23:27,512 INFO balancer.NameNodeConnector: getBlocks calls for hdfs://rc1a-dataproc-m-ewr760zc7czy40ae.mdb.yandexcloud.net:8020 will be rate-limited to 20 per second
2022-02-06 22:23:28,382 INFO balancer.Balancer: dfs.namenode.get-blocks.max-qps = 20 (default=20)
2022-02-06 22:23:28,382 IN

In [15]:
! hdfs dfs -ls /user/avito/data/

Found 1 items
-rw-r--r--   1 ubuntu hadoop 13180996366 2022-02-06 22:21 /user/avito/data/VisitsStreamNoHeader.tsv


### MapReduce

Разобъем задачу на 2 MapReduce задачи: в первой - посчитаем длины сессий для всех пользователей, во второй - отсортируем все сессии по их длине в обратном порядке и выведем топ-10. Идея алгоритма следующая:

1. Для первой MapReduce задачи делаем Маппер из изначальных данных в формат:

    ```
    UserID_1+ViewDate_11
    UserID_1+ViewDate_12
    UserID_2+ViewDate_21
    ...
    ```

2. Затем кастомный partitioner раскидывает данные по редьюсерам по ключу UserID (чтобы все строчки одного пользователя находились на 1-ой машине). Сортировка по обоим полям - сначала UserID, затем ViewDate.

3. Редьюсер группирует данные по UserID. По каждому пользователю считается длина его максимальной сессии (пользуясь тем, что ViewDate также отсортирован по возрастанию). Данные выплевываются в формате:

    ```
    UserID_1    SessionLen_1
    UserID_2    SessionLen_2
    UserID_3    SessionLen_3
    ...
    ```

4. Во второй MapReduce задаче делаем маппинг с составным ключом:

    ```
    UserID_1+SessionLen_1
    UserID_2+SessionLen_2
    UserID_3+SessionLen_3
    ...
    ```

5. Сортируем по длине сессии ($SessionLen_{i}$) в обратном порядке, затем по $UserID_{i}$

6. Редьюсер выводит первые 10 значений в формате:

    ```
    UserID_1    SessionLen_1
    UserID_2    SessionLen_2
    UserID_3    SessionLen_3
    ...
    ```

In [16]:
! rm -r ~/storage/utils
! mkdir ~/storage/utils

In [17]:
%%writefile '~/storage/utils/longest_sessions.py'

import sys
import csv
import collections
import datetime
from itertools import islice, groupby

def kv_stream(sep="\t"):
    return map(lambda x: x.split(sep), sys.stdin)

def csv_stream(sep="\t"):
    return csv.reader(iter(sys.stdin.readline, ''), delimiter=sep)

def rewind():
    collections.deque(sys.stdin, maxlen=0)

def mapper():
    for row in csv_stream():
        user_id, sess_start = row[0], row[3]
        print("{}+{}\t".format(user_id.strip(), sess_start.strip()))

def reducer():
    for user_id, sess_starts in groupby(kv_stream('+'), lambda x: x[0]):
        user_sess_starts = (x.strip() for _, x in sess_starts)

        max_sess_length = 0
        sess_length = 0
        sess = datetime.datetime.strptime(
            next(user_sess_starts), '%Y-%m-%d %H:%M:%S.%f'
        )
        for next_sess in user_sess_starts:
            next_sess = datetime.datetime.strptime(
                next_sess, '%Y-%m-%d %H:%M:%S.%f'
            )
            curr_sess_length = (next_sess - sess).total_seconds()

            if curr_sess_length <= 900.0:
                sess_length += curr_sess_length
                if sess_length > max_sess_length:
                    max_sess_length = sess_length
            else:
                sess_length = 0
            
            sess = next_sess
        
        print('{}\t{}'.format(user_id, max_sess_length))
    
def top_10_mapper():
    for k, v in kv_stream():
        print("{}+{}\t".format(k.strip(), v.strip()))

def top_10_reducer():
    first_10_stream = islice(kv_stream('+'), 10)
    
    for user, sess_len in first_10_stream:
        print("{}\t{}".format(user, sess_len.strip()))
    rewind()

if __name__ == '__main__':
    mr_command = sys.argv[1]
    {
        'map': mapper,
        'reduce': reducer,
        'top_10_map': top_10_mapper,
        'top_10_reduce': top_10_reducer,
    }[mr_command]()

Writing /home/ubuntu/storage/utils/longest_sessions.py


Запускаем 1-ую MR задачу:

In [19]:
%%time

! hdfs dfs -rm -r /user/avito/longest-sess || true
! yarn jar /usr/lib/hadoop-mapreduce/hadoop-streaming.jar \
-D mapreduce.job.name="max-session-length" \
-D mapreduce.job.reduces=3 \
-D mapreduce.job.output.key.comparator.class=org.apache.hadoop.mapreduce.lib.partition.KeyFieldBasedComparator \
-D mapreduce.partition.keycomparator.options="-k1,1 -k2,2" \
-D mapreduce.map.output.key.field.separator='+' \
-D mapred.partitioner.class=org.apache.hadoop.mapred.lib.KeyFieldBasedPartitioner \
-D mapreduce.partition.keypartitioner.options="-k1,1" \
-files ~/storage/utils/longest_sessions.py \
-mapper "python3 longest_sessions.py map" \
-reducer "python3 longest_sessions.py reduce" \
-input /user/avito/data/ \
-output /user/avito/longest-sess/

rm: `/user/avito/longest-sess': No such file or directory
packageJobJar: [] [/usr/lib/hadoop-mapreduce/hadoop-streaming-3.2.2.jar] /tmp/streamjob1376684736749830578.jar tmpDir=null
2022-02-06 22:24:59,556 INFO client.RMProxy: Connecting to ResourceManager at rc1a-dataproc-m-ewr760zc7czy40ae.mdb.yandexcloud.net/10.128.0.25:8032
2022-02-06 22:24:59,748 INFO client.AHSProxy: Connecting to Application History server at rc1a-dataproc-m-ewr760zc7czy40ae.mdb.yandexcloud.net/10.128.0.25:10200
2022-02-06 22:24:59,780 INFO client.RMProxy: Connecting to ResourceManager at rc1a-dataproc-m-ewr760zc7czy40ae.mdb.yandexcloud.net/10.128.0.25:8032
2022-02-06 22:24:59,781 INFO client.AHSProxy: Connecting to Application History server at rc1a-dataproc-m-ewr760zc7czy40ae.mdb.yandexcloud.net/10.128.0.25:10200
2022-02-06 22:25:00,041 INFO mapreduce.JobResourceUploader: Disabling Erasure Coding for path: /tmp/hadoop-yarn/staging/ubuntu/.staging/job_1644184388994_0001
2022-02-06 22:25:00,383 INFO mapred.FileIn

In [20]:
! hdfs dfs -ls /user/avito/longest-sess

Found 4 items
-rw-r--r--   1 ubuntu hadoop          0 2022-02-06 22:54 /user/avito/longest-sess/_SUCCESS
-rw-r--r--   1 ubuntu hadoop   15403255 2022-02-06 22:53 /user/avito/longest-sess/part-00000
-rw-r--r--   1 ubuntu hadoop   15405563 2022-02-06 22:53 /user/avito/longest-sess/part-00001
-rw-r--r--   1 ubuntu hadoop   15415902 2022-02-06 22:54 /user/avito/longest-sess/part-00002


In [21]:
# Макс. сессия для каждого пользователя, сек.
! hdfs dfs -cat /user/avito/longest-sess/part* | head -n 20

1000000	319.0
1000003	1932.0
1000009	978.0
1000012	1086.0
1000015	0
1000018	1488.0
1000024	848.0
1000027	987.0
1000030	4237.0
1000033	906.0
1000036	995.0
1000042	2367.0
1000045	2945.0
1000048	11772.0
100005	585.0
1000051	4849.0
1000054	2579.0
1000057	238.0
1000066	1203.0
1000069	392.0
cat: Unable to write to output stream.
cat: Unable to write to output stream.
cat: Unable to write to output stream.


2-ой MapReduce:

In [22]:
%%time

! hdfs dfs -rm -r /user/avito/top10-longest-sess || true
! yarn jar /usr/lib/hadoop-mapreduce/hadoop-streaming.jar \
-D mapreduce.job.name="top10-session-length" \
-D mapreduce.job.reduces=1 \
-D mapreduce.job.output.key.comparator.class=org.apache.hadoop.mapreduce.lib.partition.KeyFieldBasedComparator \
-D mapreduce.partition.keycomparator.options="-k2,2nr -k1,1" \
-D mapreduce.map.output.key.field.separator='+' \
-files ~/storage/utils/longest_sessions.py \
-mapper "python3 longest_sessions.py top_10_map" \
-reducer "python3 longest_sessions.py top_10_reduce" \
-input /user/avito/longest-sess/ \
-output /user/avito/top10-longest-sess/

Deleted /user/avito/top10-longest-sess
packageJobJar: [] [/usr/lib/hadoop-mapreduce/hadoop-streaming-3.2.2.jar] /tmp/streamjob2559297792847900915.jar tmpDir=null
2022-02-06 22:58:00,511 INFO client.RMProxy: Connecting to ResourceManager at rc1a-dataproc-m-ewr760zc7czy40ae.mdb.yandexcloud.net/10.128.0.25:8032
2022-02-06 22:58:00,695 INFO client.AHSProxy: Connecting to Application History server at rc1a-dataproc-m-ewr760zc7czy40ae.mdb.yandexcloud.net/10.128.0.25:10200
2022-02-06 22:58:00,722 INFO client.RMProxy: Connecting to ResourceManager at rc1a-dataproc-m-ewr760zc7czy40ae.mdb.yandexcloud.net/10.128.0.25:8032
2022-02-06 22:58:00,723 INFO client.AHSProxy: Connecting to Application History server at rc1a-dataproc-m-ewr760zc7czy40ae.mdb.yandexcloud.net/10.128.0.25:10200
2022-02-06 22:58:00,914 INFO mapreduce.JobResourceUploader: Disabling Erasure Coding for path: /tmp/hadoop-yarn/staging/ubuntu/.staging/job_1644184388994_0002
2022-02-06 22:58:01,212 INFO mapred.FileInputFormat: Total in

In [23]:
! hdfs dfs -ls /user/avito/top10-longest-sess

Found 2 items
-rw-r--r--   1 ubuntu hadoop          0 2022-02-06 22:58 /user/avito/top10-longest-sess/_SUCCESS
-rw-r--r--   1 ubuntu hadoop        162 2022-02-06 22:58 /user/avito/top10-longest-sess/part-00000


In [24]:
# Результат
! hdfs dfs -cat /user/avito/top10-longest-sess/part*

4143904	358181.0
1113291	329463.0
1203081	170447.0
4263912	80605.0
1987990	70544.0
563902	68359.0
2853587	53371.0
2735474	49881.0
1370797	49791.0
2271348	48099.0


In [26]:
! hdfs dfs -cp /user/avito/top10-longest-sess/part-00000 s3a://nsrazuvaevbucket/mhw2-top10-longest-sess.txt

2022-02-06 23:00:55,016 INFO impl.MetricsConfig: Loaded properties from hadoop-metrics2.properties
2022-02-06 23:00:55,084 INFO impl.MetricsSystemImpl: Scheduled Metric snapshot period at 10 second(s).
2022-02-06 23:00:55,084 INFO impl.MetricsSystemImpl: s3a-file-system metrics system started
2022-02-06 23:00:57,738 INFO impl.MetricsSystemImpl: Stopping s3a-file-system metrics system...
2022-02-06 23:00:57,738 INFO impl.MetricsSystemImpl: s3a-file-system metrics system stopped.
2022-02-06 23:00:57,738 INFO impl.MetricsSystemImpl: s3a-file-system metrics system shutdown complete.


Ссылка на результат: https://storage.yandexcloud.net/nsrazuvaevbucket/mhw2-top10-longest-sess.txt

## Задание 2  <a class="anchor" id="task-2"></a>

Скачаем и распакуем данные с помощью консоли: 

```
cd
kaggle competitions download -c avito-context-ad-clicks -f trainSearchStream.tsv.7z
kaggle competitions download -c avito-context-ad-clicks -f SearchInfo.tsv.7z
kaggle competitions download -c avito-context-ad-clicks -f AdsInfo.tsv.7z

mv trainSearchStream.tsv.7z storage/trainSearchStream.tsv.7z
mv SearchInfo.tsv.7z storage/SearchInfo.tsv.7z
mv AdsInfo.tsv.7z storage/AdsInfo.tsv.7z
cd storage

7z x trainSearchStream.tsv.7z -oavito-context-ad-clicks
7z x SearchInfo.tsv.7z -oavito-context-ad-clicks
7z x AdsInfo.tsv.7z -oavito-context-ad-clicks
```

In [1]:
%cd "~/storage/avito-context-ad-clicks"
! ls

/home/ubuntu/storage/avito-context-ad-clicks
AdsInfo.tsv	trainSearchStream.tsv	  VisitsStream.tsv
SearchInfo.tsv	VisitsStreamNoHeader.tsv


Запустим Spark:

In [2]:
! pip install findspark

In [3]:
import findspark
findspark.init()

In [4]:
import pyspark
from pyspark.sql import SparkSession, Row

sc = pyspark.SparkContext(appName="lsml-app")
se = SparkSession(sc)

Положим данные в hdfs:

In [21]:
%%time
! hdfs dfs -put SearchInfo.tsv /user/avito/data/
! hdfs dfs -put trainSearchStream.tsv /user/avito/data/
! hdfs dfs -put AdsInfo.tsv /user/avito/data/

CPU times: user 7.57 s, sys: 1.84 s, total: 9.41 s
Wall time: 7min 26s


In [146]:
! sudo -u hdfs hdfs balancer 

2022-02-07 08:18:04,424 INFO balancer.Balancer: namenodes  = [hdfs://rc1a-dataproc-m-ewr760zc7czy40ae.mdb.yandexcloud.net:8020]
2022-02-07 08:18:04,428 INFO balancer.Balancer: parameters = Balancer.BalancerParameters [BalancingPolicy.Node, threshold = 10.0, max idle iteration = 5, #excluded nodes = 0, #included nodes = 0, #source nodes = 0, #blockpools = 0, run during upgrade = false]
2022-02-07 08:18:04,428 INFO balancer.Balancer: included nodes = []
2022-02-07 08:18:04,428 INFO balancer.Balancer: excluded nodes = []
2022-02-07 08:18:04,429 INFO balancer.Balancer: source nodes = []
Time Stamp               Iteration#  Bytes Already Moved  Bytes Left To Move  Bytes Being Moved  NameNode
2022-02-07 08:18:04,432 INFO balancer.NameNodeConnector: getBlocks calls for hdfs://rc1a-dataproc-m-ewr760zc7czy40ae.mdb.yandexcloud.net:8020 will be rate-limited to 20 per second
2022-02-07 08:18:05,395 INFO balancer.Balancer: dfs.namenode.get-blocks.max-qps = 20 (default=20)
2022-02-07 08:18:05,395 IN

In [147]:
! hdfs dfs -ls /user/avito/data/

Found 4 items
-rw-r--r--   1 ubuntu hadoop  5350670676 2022-02-07 08:17 /user/avito/data/AdsInfo.tsv
-rw-r--r--   1 ubuntu hadoop  9469373867 2022-02-07 05:10 /user/avito/data/SearchInfo.tsv
-rw-r--r--   1 ubuntu hadoop 13180996366 2022-02-06 22:21 /user/avito/data/VisitsStreamNoHeader.tsv
-rw-r--r--   1 ubuntu hadoop 11023566785 2022-02-07 05:16 /user/avito/data/trainSearchStream.tsv


### Пункт 1  <a class="anchor" id="task-2-1"></a>

In [31]:
searchInfoDF = se.read.csv("/user/avito/data/SearchInfo.tsv", header=True, inferSchema=True, sep='\t')
searchInfoDF.printSchema()

root
 |-- SearchID: integer (nullable = true)
 |-- SearchDate: string (nullable = true)
 |-- IPID: integer (nullable = true)
 |-- UserID: integer (nullable = true)
 |-- IsUserLoggedOn: integer (nullable = true)
 |-- SearchQuery: string (nullable = true)
 |-- LocationID: integer (nullable = true)
 |-- CategoryID: integer (nullable = true)
 |-- SearchParams: string (nullable = true)



In [52]:
import re

def extract_categories(text):
    """
    Вытаскиваем номер категории рег. выражением
    """
    regexp = r'(\d+):'
    return re.findall(regexp, text)

In [63]:
%%time

# Без сохранения в hdfs
top10cat = (
    searchInfoDF.rdd
    # Убираем пропуски
    .filter(lambda x: x.SearchParams != None)
    # Достаем категории
    .flatMap(lambda x: extract_categories(x.SearchParams))
    # Считаем
    .map(lambda x: (x, 1))
    .reduceByKey(lambda a, b: a + b)
    .takeOrdered(10, lambda x: -x[1])
)
top10cat

CPU times: user 55.3 ms, sys: 25.6 ms, total: 80.9 ms
Wall time: 4min 54s


[('175', 9979618),
 ('5', 8737931),
 ('178', 6378281),
 ('45', 6375642),
 ('83', 6351296),
 ('127', 5196744),
 ('179', 3245870),
 ('143', 3108130),
 ('709', 2684001),
 ('44', 2646546)]

In [61]:
%%time

# С автосохранением в hdfs
! hdfs dfs -rm -r /user/avito/top10-categories
(
    searchInfoDF.rdd
    # Убираем пропуски
    .filter(lambda x: x.SearchParams != None)
    # Достаем категории
    .flatMap(lambda x: extract_categories(x.SearchParams))
    # Считаем
    .map(lambda x: (x, 1))
    .reduceByKey(lambda a, b: a + b)
    # Сортируем для сохранения
    .map(lambda x: (x[1], x[0]))
    .sortByKey(ascending=False)
    .zipWithIndex()
    .filter(lambda x: x[1] < 10)
    .map(lambda x: x[0])
    .saveAsTextFile('/user/avito/top10-categories')
)

CPU times: user 98.3 ms, sys: 58.3 ms, total: 157 ms
Wall time: 5min 13s


In [62]:
! hdfs dfs -cat /user/avito/top10-categories/*

(9979618, '175')
(8737931, '5')
(6378281, '178')
(6375642, '45')
(6351296, '83')
(5196744, '127')
(3245870, '179')
(3108130, '143')
(2684001, '709')
(2646546, '44')


In [90]:
# Объединим файлы в 1
! hdfs dfs -rm -r /user/avito/top10-categories-merged
(
    sc.textFile("/user/avito/top10-categories/*")
    .map(lambda x: re.findall(r'\d+', x))
    .map(lambda x: '\t'.join(x[::-1]))
    .coalesce(1)
    .saveAsTextFile('/user/avito/top10-categories-merged')
)

Deleted /user/avito/top10-categories-merged


In [91]:
! hdfs dfs -ls /user/avito/top10-categories-merged

Found 2 items
-rw-r--r--   1 ubuntu hadoop          0 2022-02-07 06:46 /user/avito/top10-categories-merged/_SUCCESS
-rw-r--r--   1 ubuntu hadoop        115 2022-02-07 06:46 /user/avito/top10-categories-merged/part-00000


In [92]:
! hdfs dfs -cat /user/avito/top10-categories-merged/*

175	9979618
5	8737931
178	6378281
45	6375642
83	6351296
127	5196744
179	3245870
143	3108130
709	2684001
44	2646546


In [94]:
! hdfs dfs -cp /user/avito/top10-categories-merged/part-00000 s3a://nsrazuvaevbucket/mhw2-top10-categories.txt

2022-02-07 06:47:37,724 INFO impl.MetricsConfig: Loaded properties from hadoop-metrics2.properties
2022-02-07 06:47:37,795 INFO impl.MetricsSystemImpl: Scheduled Metric snapshot period at 10 second(s).
2022-02-07 06:47:37,795 INFO impl.MetricsSystemImpl: s3a-file-system metrics system started
2022-02-07 06:47:41,902 INFO impl.MetricsSystemImpl: Stopping s3a-file-system metrics system...
2022-02-07 06:47:41,902 INFO impl.MetricsSystemImpl: s3a-file-system metrics system stopped.
2022-02-07 06:47:41,902 INFO impl.MetricsSystemImpl: s3a-file-system metrics system shutdown complete.


Ссылка на результат: https://storage.yandexcloud.net/nsrazuvaevbucket/mhw2-top10-categories.txt

### Пункт 2 <a class="anchor" id="task-2-2"></a>

In [97]:
trainSearchStreamDF = se.read.csv(
    "/user/avito/data/trainSearchStream.tsv", header=True,
    inferSchema=True, sep='\t'
)
trainSearchStreamDF.printSchema()

root
 |-- SearchID: integer (nullable = true)
 |-- AdID: integer (nullable = true)
 |-- Position: integer (nullable = true)
 |-- ObjectType: integer (nullable = true)
 |-- HistCTR: double (nullable = true)
 |-- IsClick: integer (nullable = true)



In [100]:
searchInfoDF.printSchema()

root
 |-- SearchID: integer (nullable = true)
 |-- SearchDate: string (nullable = true)
 |-- IPID: integer (nullable = true)
 |-- UserID: integer (nullable = true)
 |-- IsUserLoggedOn: integer (nullable = true)
 |-- SearchQuery: string (nullable = true)
 |-- LocationID: integer (nullable = true)
 |-- CategoryID: integer (nullable = true)
 |-- SearchParams: string (nullable = true)



In [99]:
searchInfoDF.registerTempTable('searchInfo')
trainSearchStreamDF.registerTempTable('trainSearchStream')

In [122]:
# Склеим таблицы по SearchID и уберем пропуски
searchQueryDF = se.sql(
    """
    SELECT a.SearchID, a.SearchQuery, b.IsClick
    FROM 
        searchInfo a INNER JOIN trainSearchStream b 
        ON a.SearchID = b.SearchID
    WHERE 
        IsClick = 1 AND SearchQuery IS NOT NULL
    """
).cache()

In [124]:
%%time

searchQueryDF.rdd.take(10)

CPU times: user 59.1 ms, sys: 16.2 ms, total: 75.3 ms
Wall time: 7min 40s


[Row(SearchID=34239, SearchQuery='монопод', IsClick=1),
 Row(SearchID=90019, SearchQuery='сварочный аппарат бу', IsClick=1),
 Row(SearchID=96488, SearchQuery='iphone 5s новый', IsClick=1),
 Row(SearchID=150051, SearchQuery='ваз 2110', IsClick=1),
 Row(SearchID=154452, SearchQuery='туфли', IsClick=1),
 Row(SearchID=340002, SearchQuery='счетчик', IsClick=1),
 Row(SearchID=419385, SearchQuery='микроволновая печь', IsClick=1),
 Row(SearchID=469345, SearchQuery='мокасины', IsClick=1),
 Row(SearchID=564376, SearchQuery='продажа велосипеда', IsClick=1),
 Row(SearchID=567690, SearchQuery='бампер ниссан примера', IsClick=1)]

In [132]:
import re

def extract_words(text):
    pattern = re.compile(r"[a-zа-я]+")
    result = []
    for match in pattern.finditer(text.lower()):
        word = match.group(0)
        result.append(word)
    return result

In [142]:
%%time

# С автосохранением в hdfs
! hdfs dfs -rm -r /user/avito/top10-words
(
    searchQueryDF.rdd
    # Достаем слова из запросов
    .flatMap(lambda x: extract_words(x.SearchQuery))
    # Считаем
    .map(lambda x: (x, 1))
    .reduceByKey(lambda a, b: a + b)
    # Сортируем и сохраняем
    .map(lambda x: (x[1], x[0]))
    .sortByKey(ascending=False)
    .zipWithIndex()
    .filter(lambda x: x[1] < 10)
    .coalesce(1)
    .map(lambda x: '\t'.join(map(str, x[0][::-1])))
    .saveAsTextFile('/user/avito/top10-words')
)

rm: `/user/avito/top10-words': No such file or directory
CPU times: user 156 ms, sys: 82.6 ms, total: 239 ms
Wall time: 32.4 s


In [143]:
! hdfs dfs -cat /user/avito/top10-words/part*

велосипед	12510
бу	11211
iphone	7439
для	5984
диван	5734
s	5571
на	4905
платье	4471
коляска	3794
велосипеды	3780


In [144]:
! hdfs dfs -cp /user/avito/top10-words/part-00000 s3a://nsrazuvaevbucket/mhw2-top10-words.txt

2022-02-07 08:03:04,368 INFO impl.MetricsConfig: Loaded properties from hadoop-metrics2.properties
2022-02-07 08:03:04,439 INFO impl.MetricsSystemImpl: Scheduled Metric snapshot period at 10 second(s).
2022-02-07 08:03:04,439 INFO impl.MetricsSystemImpl: s3a-file-system metrics system started
2022-02-07 08:03:07,181 INFO impl.MetricsSystemImpl: Stopping s3a-file-system metrics system...
2022-02-07 08:03:07,182 INFO impl.MetricsSystemImpl: s3a-file-system metrics system stopped.
2022-02-07 08:03:07,182 INFO impl.MetricsSystemImpl: s3a-file-system metrics system shutdown complete.


Ссылка на результат: https://storage.yandexcloud.net/nsrazuvaevbucket/mhw2-top10-words.txt

### Пункт 3 <a class="anchor" id="task-2-3"></a>

In [148]:
adsInfoDF = se.read.csv(
    "/user/avito/data/AdsInfo.tsv", header=True,
    inferSchema=True, sep='\t'
)
adsInfoDF.printSchema()

root
 |-- AdID: integer (nullable = true)
 |-- LocationID: integer (nullable = true)
 |-- CategoryID: integer (nullable = true)
 |-- Params: string (nullable = true)
 |-- Price: double (nullable = true)
 |-- Title: string (nullable = true)
 |-- IsContext: integer (nullable = true)



In [149]:
adsInfoDF.registerTempTable('adsInfo')

In [173]:
import re

def extract_words_with_price(text, price):
    pattern = re.compile(r"[a-zа-я]+")
    result = []
    for match in pattern.finditer(text.lower()):
        word = match.group(0)
        result.append(word)
    return zip(result, len(result)*[price])

def find_mean(key, group):
    group = list(group)
    return key, sum(group) / len(group)

In [174]:
%%time

# С автосохранением в hdfs
! hdfs dfs -rm -r /user/avito/top10-mean-price
(
    adsInfoDF.rdd
    # Убираем пропуски
    .filter(lambda x: (x.Title != None) and (x.Price != None))
    # Достаем слова с их стоимостью
    .flatMap(lambda x: extract_words_with_price(x.Title, x.Price))
    # Групируем по словам, считая среднее
    .groupByKey()
    .map(lambda x: find_mean(x[0], x[1]))
    # Сортируем и сохраняем
    .map(lambda x: (x[1], x[0]))
    .sortByKey(ascending=False)
    .zipWithIndex()
    .filter(lambda x: x[1] < 10)
    .coalesce(1)
    .map(lambda x: '\t'.join(map(str, x[0][::-1])))
    .saveAsTextFile('/user/avito/top10-mean-price')
)

rm: `/user/avito/top10-mean-price': No such file or directory
CPU times: user 110 ms, sys: 54.2 ms, total: 164 ms
Wall time: 4min 4s


In [175]:
! hdfs dfs -cat /user/avito/top10-mean-price/part*

екуеу	999999999999.0
вазя	999999999999.0
москвском	999999999999.0
аваыаыв	888888888888.0
rytrytry	777777777777.0
красавы	500000000249.5
строимград	500000000000.0
универсамы	450000008000.0
почивники	144781144847.66666
стима	119870533654.83333


In [176]:
! hdfs dfs -cp /user/avito/top10-mean-price/part-00000 s3a://nsrazuvaevbucket/mhw2-mean-price.txt

2022-02-07 08:59:14,867 INFO impl.MetricsConfig: Loaded properties from hadoop-metrics2.properties
2022-02-07 08:59:14,948 INFO impl.MetricsSystemImpl: Scheduled Metric snapshot period at 10 second(s).
2022-02-07 08:59:14,948 INFO impl.MetricsSystemImpl: s3a-file-system metrics system started
2022-02-07 08:59:17,951 INFO impl.MetricsSystemImpl: Stopping s3a-file-system metrics system...
2022-02-07 08:59:17,951 INFO impl.MetricsSystemImpl: s3a-file-system metrics system stopped.
2022-02-07 08:59:17,951 INFO impl.MetricsSystemImpl: s3a-file-system metrics system shutdown complete.


Ссылка на результат: https://storage.yandexcloud.net/nsrazuvaevbucket/mhw2-mean-price.txt

### Пункт 4 <a class="anchor" id="task-2-4"></a>

In [5]:
visitsStreamDF = se.read.csv(
    "/user/avito/data/VisitsStreamNoHeader.tsv", header=False,
    inferSchema=True, sep='\t'
)
columns = [
    'UserID',
    'IPID',
    'AdID',
    'ViewDate'
]
visitsStreamDF = visitsStreamDF.toDF(*columns)
visitsStreamDF.show(10)

+------+-------+--------+--------------------+
|UserID|   IPID|    AdID|            ViewDate|
+------+-------+--------+--------------------+
| 59703|1259356|  469877|2015-04-25 00:00:...|
|154389|1846749|27252551|2015-04-25 00:00:...|
|218628|2108380|31685325|2015-04-25 00:00:...|
|231535| 837110|18827716|2015-04-25 00:00:...|
|282306|1654210|29363673|2015-04-25 00:00:...|
|295068| 601505|  588324|2015-04-25 00:00:...|
|501897| 158476| 4103261|2015-04-25 00:00:...|
|655394| 631692| 9860544|2015-04-25 00:00:...|
|765603| 804403|29475627|2015-04-25 00:00:...|
|790289| 121085|23309988|2015-04-25 00:00:...|
+------+-------+--------+--------------------+
only showing top 10 rows



In [6]:
import datetime

start_date = datetime.datetime.strptime(
    visitsStreamDF.head(1)[0].ViewDate, 
    '%Y-%m-%d %H:%M:%S.%f'
)
end_date = datetime.datetime.strptime(
    visitsStreamDF.tail(1)[0].ViewDate,
    '%Y-%m-%d %H:%M:%S.%f'
)
numdays = (end_date - start_date).days + 1
print(start_date, end_date)
print(numdays)

def check_activity(key, group):
    """
    За время наблюдения прошло 26 дней.
    Если пользователь заходил в каждый из них, то
    уникальных дней как раз 26.
    """
    if len(set(group)) == 26:
        return key

2015-04-25 00:00:00 2015-05-20 17:59:51
26


In [12]:
%%time

all_days_active_users = (
    visitsStreamDF.rdd
    .map(lambda x: (x.UserID, x.ViewDate[:10]))
    # Группируем по пользователям, возвращая UserID тех,
    # кто заходил каждый день
    .groupByKey()
    .map(lambda x: check_activity(x[0], x[1]))
    .collect()
)
all_days_active_users[:10]

CPU times: user 148 ms, sys: 80.2 ms, total: 228 ms
Wall time: 14min 25s


[None, None, None, None, None, None, None, None, None, None]

In [19]:
# Убираем None
all_days_active_users = [i for i in all_days_active_users if i != None]
print(all_days_active_users[:10])
print('Всего пользователей, заходивших каждый день:', len(all_days_active_users))

[682902, 1633302, 344025, 2585682, 985842, 3809718, 851202, 1758438, 590733, 3165526]
Всего пользователей, заходивших каждый день: 807


In [29]:
# Сохраним результат на всякий случай
! mkdir other
with open('other/active-users.txt', 'w') as f:
    for i in all_days_active_users:
        f.write(str(i) + '\n')

In [24]:
with open('other/all-days-active-users.txt', 'w') as f:
    f.write(str(len(all_days_active_users)))

! hdfs dfs -put other/all-days-active-users.txt s3a://nsrazuvaevbucket/mhw2-all-days-active-users.txt

2022-02-07 20:26:19,881 INFO impl.MetricsConfig: Loaded properties from hadoop-metrics2.properties
2022-02-07 20:26:19,949 INFO impl.MetricsSystemImpl: Scheduled Metric snapshot period at 10 second(s).
2022-02-07 20:26:19,949 INFO impl.MetricsSystemImpl: s3a-file-system metrics system started
2022-02-07 20:26:23,401 INFO impl.MetricsSystemImpl: Stopping s3a-file-system metrics system...
2022-02-07 20:26:23,402 INFO impl.MetricsSystemImpl: s3a-file-system metrics system stopped.
2022-02-07 20:26:23,402 INFO impl.MetricsSystemImpl: s3a-file-system metrics system shutdown complete.


Ссылка на результат: https://storage.yandexcloud.net/nsrazuvaevbucket/mhw2-all-days-active-users.txt

### Пункт 5 <a class="anchor" id="task-2-5"></a>

In [37]:
def uniq_users_per_day(key, group):
    return key, len(set(group))

In [39]:
%%time

(
    visitsStreamDF.rdd
    .map(lambda x: (x.ViewDate[:10], x.UserID))
    # Убираем пользователей, заходивших каждый день
    .filter(lambda x: x[1] not in all_days_active_users)
    # Группируем по дате, считая кол-во уникальных пользователей
    .groupByKey()
    .map(lambda x: uniq_users_per_day(x[0], x[1]))
    # Сортируем и сохраняем
    .map(lambda x: (x[1], x[0]))
    .sortByKey(ascending=False)
    .zipWithIndex()
    .map(lambda x: '\t'.join(map(str, x[0][::-1])))
    .coalesce(1)
    .saveAsTextFile('/user/avito/uniq-users-per-day')
)

CPU times: user 225 ms, sys: 102 ms, total: 327 ms
Wall time: 25min 58s


In [40]:
! hdfs dfs -cat /user/avito/uniq-users-per-day/part-00000

2015-05-12	973248
2015-05-13	847446
2015-05-14	745904
2015-05-06	733027
2015-05-05	731624
2015-05-07	729708
2015-05-11	706240
2015-04-27	676433
2015-05-08	664982
2015-04-28	655791
2015-04-29	641148
2015-05-15	633132
2015-05-04	607205
2015-04-30	596224
2015-05-10	562151
2015-04-26	540051
2015-05-03	511687
2015-04-25	498069
2015-05-16	493577
2015-05-18	488250
2015-05-17	480968
2015-05-02	468010
2015-05-01	452800
2015-05-09	450135
2015-05-19	367777
2015-05-20	184972


In [41]:
! hdfs dfs -cp /user/avito/uniq-users-per-day/part-00000 s3a://nsrazuvaevbucket/mhw2-uniq-users-per-day.txt

2022-02-07 21:04:33,361 INFO impl.MetricsConfig: Loaded properties from hadoop-metrics2.properties
2022-02-07 21:04:33,428 INFO impl.MetricsSystemImpl: Scheduled Metric snapshot period at 10 second(s).
2022-02-07 21:04:33,429 INFO impl.MetricsSystemImpl: s3a-file-system metrics system started
2022-02-07 21:04:36,189 INFO impl.MetricsSystemImpl: Stopping s3a-file-system metrics system...
2022-02-07 21:04:36,189 INFO impl.MetricsSystemImpl: s3a-file-system metrics system stopped.
2022-02-07 21:04:36,190 INFO impl.MetricsSystemImpl: s3a-file-system metrics system shutdown complete.


Ссылка на результат: https://storage.yandexcloud.net/nsrazuvaevbucket/mhw2-uniq-users-per-day.txt